In [2]:
## imporrt libarary 
import os
import pandas as pd

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
# from google.cloud import aiplatform
# from support_functions import missing_value, fill_missing, list_dtypes

## sklearn module
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import pickle


In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "trial_bigq.json"

In [4]:
project_id = 'dtidsUS'
dataset_id = 'capstone'
table_id = 'data_saudi_used_cars'
region = 'us-central1'
bucket_name = 'modul4'
blob_name = 'saini/data_saudi_used_cars.csv'

In [5]:
client = bigquery.Client(project='dtidsus')

In [6]:
try : 
  storage_client = storage.Client(project=project_id)
  bucket = storage_client.get_bucket(bucket_name) # Add bucket name
  data_capstone = bucket.blob('saini/data_saudi_used_cars.csv')
  data_capstone.upload_from_filename('dataset/data_saudi_used_cars.csv')

  print ("Uploading model succeeded")
except:
  raise TypeError("An exception occurred")

Uploading model succeeded


In [7]:
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")

In [8]:
df = query_job.result().to_dataframe()

c:\SANS\knowledge\PURWADHIKA\capstone\capstone 3\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [9]:
df.duplicated().sum()

np.int64(4)

In [10]:
df.isna().sum()

Type           0
Region         0
Make           0
Gear_Type      0
Origin         0
Options        0
Year           0
Engine_Size    0
Mileage        0
Negotiable     0
Price          0
dtype: int64

In [11]:
df.head(5)

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Price
0,2,Riyadh,Mazda,Automatic,Saudi,Standard,2011,1.5,37000,False,14000
1,3,Al-Medina,Mazda,Automatic,Saudi,Standard,2018,1.8,53000,False,54000
2,3,Al-Medina,Mazda,Automatic,Saudi,Standard,2018,2.0,31000,False,54000
3,3,Al-Medina,Mazda,Automatic,Saudi,Standard,2018,2.0,47000,False,54000
4,3,Riyadh,Mazda,Automatic,Saudi,Semi Full,2016,2.0,104800,False,39000


**Upload Model to GCP**

In [12]:
# upload model
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob('saini/model/model.pkl')
    blob_model.upload_from_filename('Saudi-Used-Cars-XGB-ML-Regression-Model.pkl')

    print ("Uploading model succeeded")
except:
    raise TypeError("An exception occurred")



Uploading model succeeded


In [ ]:
from google.cloud import aiplatform

In [13]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "dev_trial.json"

In [ ]:
aiplatform.init(project='dtidsus', location='us-central1')

model = aiplatform.Model.upload(
  display_name='kevin_model_000',
  artifact_uri="gs://modul4/saini/model/",
  serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

In [ ]:
endpoint = aiplatform.Endpoint.create(
  display_name="jaya-endpoint-000",
  project='dtidsus',
  location='us-central1',
)

In [ ]:
min_replica_count: int = 1
max_replica_count: int = 1

In [ ]:
endpoint.deploy( 
  model=model,
  deployed_model_display_name='jaya_model_000',
  machine_type='e2-standard-2',
  min_replica_count=min_replica_count,
  max_replica_count=max_replica_count,
  sync=True, 
)

**Using Endpoint**

In [ ]:
## predict your data with online prediction here 
PROJECT_ID = 'dtidsus'
ENDPOINT_ID = "projects/41965541199/locations/us-central1/endpoints/2241324766407426048"
REGION = 'us-central1'

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)
endpoint = aiplatform.Endpoint(ENDPOINT_ID)
prediction = endpoint.predict(instances=[[50000, 1, 3, 1, 23, 131895, 132154, 129237]])

print("PREDICTION:", prediction)

**Manual File**

In [ ]:
try:
  storage_client = storage.Client(project=project_id)
  bucket = storage_client.get_bucket(bucket_name)
  blob_model = bucket.blob('ilham/model/model.pkl')
  blob_model.download_to_filename('model_new.pkl')

  print ("download model succeeded")
except:
  raise TypeError("An exception occurred")

In [ ]:
model_load = pickle.load(open('model_new.pkl','rb'))

In [ ]:
y_predict = model_load.predict(X_test)

In [ ]:
y_predict

In [ ]:
import category_encoders as ce